# Extract Sentences

`COLX_523_Group7/env`: work in the local `virtualenv`!!!

In [12]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/COLX_523')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Getting Started

In [73]:
import time
import spacy
import numpy as np
import pandas as pd
import pickle
import os.path as path
import csv
import random, os, json, csv
random.seed(523)
from collections import Counter

import nltk
nltk.download('stopwords', quiet=True) | nltk.download('punkt', quiet=True)
from nltk import sent_tokenize, word_tokenize
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
import pandas as pd

In [2]:
nlp = spacy.load('en_core_web_lg')

In [3]:
def load_master(master_dir):
    """Returns three master files present in `master_dir`. If such directory does
    not exist, create one and three master files, and then return them. 
    """    
    with open(os.path.join(master_dir, 'submission.json'), 'r', encoding='utf-8') as f:
        submission_dict = json.load(f)
    with open(os.path.join(master_dir, 'delta_thread.json'), 'r', encoding='utf-8') as f:
        delta_thread_dict = json.load(f)
    with open(os.path.join(master_dir, 'log.json'), 'r') as f:
        log_dict = json.load(f)

    return submission_dict, delta_thread_dict, log_dict

In [4]:
MASTER_DIR = '../src/reddit_scraper/reddit_corpus_mar5'  #'feb26'
submission_dict, delta_thread_dict, log_dict = load_master(MASTER_DIR)

# Extract `climate change` submissions and threads

In [5]:
KEYWORD = 'climate change'
s_df = pd.DataFrame(submission_dict)
cc_idx = s_df[s_df['submission_title'].str.contains(KEYWORD, case=False)].index
cc_submission_ids = s_df.loc[cc_idx]['submission_id']
#print(cc_submission_ids.values)  # f0y5af  g0tkib  p7h645
cc_thread = []
for key in delta_thread_dict:
    s_id, _ = key.split('-')
    if s_id in cc_submission_ids.values:
        cc_thread.append(key)
print(len(cc_submission_ids), 'topics', len(cc_thread), 'threads')  # 15 (feb26)  --> 41

76 topics 117 threads


In [6]:
def get_submission_title(submission_id):
    idx = submission_dict['submission_id'].index(submission_id)
    return submission_dict['submission_title'][idx]

In [7]:
def get_submission_body(submission_id):
    idx = submission_dict['submission_id'].index(submission_id)
    return submission_dict['submission_body'][idx]

In [8]:
comment = delta_thread_dict['f0y5af-fh0f5n6'][0]
comment

{'comment_id': 'fgzuyrk',
 'created_utc': 1581197912.0,
 'timestamp': '2020-02-08 21:38:32',
 'author': 'yyzjertl',
 'parent_id': 't3_f0y5af',
 'comment': 'I mean, I know several Trump supporters who believe climate change is happening and is human caused, and think what Trump says about this issue is technically wrong.'}

In [11]:
doc = nlp(comment['comment'])
for sent in doc.sents:
    print(sent)  # Span object
    print(sent.text)  # str
    print(sent.text.split())  # simple split
    nltk = word_tokenize(sent.text)   # NLTK tokenizer
    print(len(nltk), nltk)

I mean, I know several Trump supporters who believe climate change is happening and is human caused, and think what Trump says about this issue is technically wrong.
I mean, I know several Trump supporters who believe climate change is happening and is human caused, and think what Trump says about this issue is technically wrong.
['I', 'mean,', 'I', 'know', 'several', 'Trump', 'supporters', 'who', 'believe', 'climate', 'change', 'is', 'happening', 'and', 'is', 'human', 'caused,', 'and', 'think', 'what', 'Trump', 'says', 'about', 'this', 'issue', 'is', 'technically', 'wrong.']
31 ['I', 'mean', ',', 'I', 'know', 'several', 'Trump', 'supporters', 'who', 'believe', 'climate', 'change', 'is', 'happening', 'and', 'is', 'human', 'caused', ',', 'and', 'think', 'what', 'Trump', 'says', 'about', 'this', 'issue', 'is', 'technically', 'wrong', '.']


In [12]:
nltk = word_tokenize(comment['comment'])
print(len(word_tokenize(sent.text)))

31


# Word counts of climate change texts

In [68]:
def get_title_tokens(submission_id):
    text = get_submission_title(submission_id)
    sents = sent_tokenize(text)
    words = word_tokenize(text)
    return sents, words

def get_body_tokens(submission_id):
    text = get_submission_body(submission_id)
    sents = sent_tokenize(text)
    words = word_tokenize(text)
    return sents, words

def count_corpus(submission_ids, thread_ids):
    sub_sent_count, sub_word_count = 0, 0  # submissions
    thr_sent_count, thr_word_count = 0, 0  # threads

    for submission_id in submission_ids:
        sents, words = get_title_tokens(submission_id)
        sub_sent_count += len(sents)
        sub_word_count += len(words)
        sents, words = get_body_tokens(submission_id)
        sub_sent_count += len(sents)
        sub_word_count += len(words)
    print('Submissions:', sub_sent_count, 'sentences', sub_word_count, 'words')

    for thread_id in thread_ids:
        for comment_dict in delta_thread_dict[thread_id]:
            comment = comment_dict['comment'].replace('\n\n', ' ')
            sents = sent_tokenize(comment)
            words = word_tokenize(comment)
            thr_sent_count += len(sents)
            thr_word_count += len(words)
    print('Threads:', thr_sent_count, 'sentences', thr_word_count, 'words')
    print('-'*50, )
    print('Total:', sub_sent_count+thr_sent_count, 'sentences', sub_word_count+thr_word_count, 'words')

In [72]:
# Entire corpus
count_corpus(submission_dict['submission_id'], delta_thread_dict.keys())

Submissions: 45322 sentences 997521 words
Threads: 62386 sentences 1238032 words
--------------------------------------------------
Total: 107708 sentences 2235553 words


In [69]:
# Climate Change text
count_corpus(cc_submission_ids, cc_thread)

Submissions: 1354 sentences 32338 words
Threads: 3170 sentences 65826 words
--------------------------------------------------
Total: 4524 sentences 98164 words


# TfIdf
https://www.geeksforgeeks.org/tf-idf-for-bigrams-trigrams/

https://qiita.com/fujin/items/b1a7152c2ec2b4963160

1. Create a list of word tokenized sentences
2. Preprocessing & stopwords removal
3. Tf-Idf

In [75]:
def remove_invalid_words(sent):
    stop_words = set(stopwords.words('english'))
    invalid_words = ['cmv', '&#x200B;', 'delta!', 'xb', '∆', ':)']
    return ' '.join([x for 
            x in nltk.word_tokenize(sent) if 
            (x not in stop_words) and (x not in invalid_words)])

In [76]:
# Preprocessing
def remove_string_special_characters(sent):
    # removes special characters with ' '
    stripped = re.sub('[^a-zA-z\s]', '', sent)
    stripped = re.sub('_', '', stripped)
      
    # Change any white space to one space
    stripped = re.sub('\s+', ' ', stripped)
      
    # Remove start and end white spaces
    stripped = stripped.strip()
    if stripped != '':
        return stripped.lower()
    else:
#        print("NONE!!!", sent)
        return None

In [77]:
def clean_up_text(text):
    tokenized = []
    clean = text.replace('\n\n', ' ')
    try:
        doc = nlp(clean)
        for sent in doc.sents:
            clean = remove_string_special_characters(sent.text)
            if clean:
                clean = remove_invalid_words(clean)
            if clean:
                tokenized.append(clean)
        return tokenized
    except Exception as e:
        return None        

In [78]:
txt1 = []  # title
for submission_id in cc_submission_ids.values:
    text = get_submission_title(submission_id)
    txt1.extend(clean_up_text(text))   
print(len(txt1), 'sentences in', len(cc_submission_ids), 'submissions')

83 sentences in 76 submissions


In [79]:
txt2 = []  # body
for submission_id in cc_submission_ids.values:
    text = get_submission_body(submission_id)
    txt2.extend(clean_up_text(text))
print(len(txt2), 'sentences in', len(cc_submission_ids), 'submission bodies')

1287 sentences in 76 submission bodies


In [80]:
txt3 = []  # threads
for thread_id in cc_thread:
    for comment_dict in delta_thread_dict[thread_id]:
        text = comment_dict['comment']
        txt3.extend(clean_up_text(text))   
print(len(txt3), 'sentences in', len(cc_thread), 'threads')

3227 sentences in 117 threads


In [81]:
all_txt = txt1 + txt2 + txt3
len(all_txt)

4597

## Unigrams & Bigrams

In [84]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=3)
X = vectorizer.fit_transform(all_txt)
features = (vectorizer.get_feature_names())
sums = X.sum(axis=0)
data = []
for column, term in enumerate(features):
    data.append((term, sums[0, column]))
ranking = pd.DataFrame(data, columns=['term', 'rank'])
ranked_words = ranking.sort_values('rank', ascending=False)
print(ranked_words.shape, '\n', ranked_words.head(20))

(3660, 2) 
                 term        rank
455           change  105.214976
544          climate  100.230036
547   climate change   86.551904
2334          people   78.095081
3598           would   67.411036
3264           think   59.727931
907             dont   53.041031
1867            like   49.614273
814            delta   49.576591
2248             one   41.960582
1083            even   36.553922
2428           point   36.011167
3238           thats   35.629433
1365             get   35.572902
3580           world   35.391770
116             also   35.147767
2118            much   34.725528
1030          energy   33.792855
3418              us   32.780142
1961            make   31.957445


## Save on the disk

In [87]:
with open('tfidf_result.txt', 'w') as f:
    count = 0
    for term, rank in zip(ranked_words['term'][count:], ranked_words['rank'][count:]):
        f.write(f'{count}\t{term}\t{round(rank, 2)}\n')
        count += 1

# Extract the sentences for annotation

In [88]:
# 1: Extract all sentences from the climate exchange topics
# 2. Iterate over the list of "climate change" thread_ids
# 3. For each comment under the thread, apply spaCy tokenizer (comment -> sentence)
# 4. Add the tokenized comment (a set of sentences) to CSV, along with 

In [57]:
def remove_invalid_words(sent):
    invalid_words = ['cmv', '&#x200B;', 'delta!', 'xb', '∆', ':)']
    return ' '.join([x for 
            x in nltk.word_tokenize(sent) if 
            x not in invalid_words])

# Preprocessing
def preprocess(sent):

    # Change any white space to one space
    clean = re.sub('\s+', ' ', sent).strip()
      
    # Remove start and end white spaces
    try:
        if clean != '':
            clean = remove_invalid_words(clean)
            if clean != '':
                return clean
    except:
        print("NONE!!!", sent)
        return None

In [51]:
KEY_MUST = ['pollution', 'protest', 'responsible', 'green energy', 'emission', 'dishonest', 
            'change', 'issue', 'fact', 'control', 'viable', 'solve', 'health', 'compromise', 
            'order', 'answer', 'effect', 'measures', 'nuclear energy', 'disaster', 
            'actively deny', 'address', 'fossil fuel', 'responsibilit', 'global climate', 
            'worry', 'renewable', 'cause', 'problem', 'global', 'co2', 'society', 
            'carbon footprint', 'discuss', 'nuclear', 'solution', 'climate change', 'atmospher', 
            'prevent', 'ozone hole', 'carbon'
           ]

KEY_ASPECT = {
    'politics': ['policy', 'developed countr', 'socialism', 'bill', 'developing countr', 
                 'politician', 'conservative', 'countr', 'politicize', 'government', 
                 'country', 'republican', 'right wing', 'democrat', 'state'
    ],
    'humanity': ['change myth', 'consumer', 'extinction', 'stewardship', 'students', 'wealth', 
                 'children', 'end human', 'extinct', 'human civilization', 'generation', 
                 'kids', 'mancaused', 'apocalypse', 'humanity', 'apocalyptic', 'existential threat', 
                 'luxuries', 'education', 'healthcare', 'lifestyle', 'overpopulation', 'citizen', 
                 'society', 'starvation', 'moral', 'selfish', 'everyday people', 
                 'individual responsibilit', 'individuals', 'survive', 'poverty', 'people', 
                 'manmade', 'human', 'people believe', 'activist', 'civilization', 'habitable', 
                 'spending'
    ],
    'science': ['academ', 'advancement', 'experiment', 'scientist', 'academia', 
                'technological', 'renewable', 'green', 'innovation', 'medical', 'technology', 
                'scientific', 'empirical', 'science', 'research', 'scholar'
    ],
    'industry': ['corporation', 'invest', 'ceo', 'market', 'ethic', 'industry', 'economy', 
                 'company', 'entity', 'profit'
    ]
}

def contain_must_key(sent):
    flag = False
    for keyword in KEY_MUST:
        if keyword in sent:
            flag = True
    return flag

def contain_aspect_key(sent, aspect):
    flag = False
    for keyword in KEY_ASPECT[aspect]:
        if keyword in sent:
            flag = True
    return flag

def contain_keyword(sent, aspect):
    flag = False
    if contain_must_key(sent):
        if contain_aspect_key(sent, aspect):
            flag = True
    return flag

def get_keyword(sent, aspect):
    keywords = []
    for keyword in KEY_MUST+KEY_ASPECT[aspect]:
        if keyword in sent:
            keywords.append(keyword)
#    print(keywords)
    return keywords

1. Initialize selected_sent_ids (next cell)
2. L7-8 --> choose 'w' or 'a'
3. L10 --> comment off from the 2nd run

In [60]:
selected_sent_ids = []

In [64]:
MIN, MAX = 15, 70 # a tweet 280 characters/avg 5 char per word = 56
aspect_count = 0
#ASPECT = 'politics'
#ASPECT = 'humanity'
#ASPECT = 'science'
ASPECT = 'industry'
#with open('climate_change_raw_v3.csv', 'w', newline='') as f:  # for 1st run
with open('climate_change_raw_v3.csv', 'a', newline='') as f:  # for 2nd onwards
    writer = csv.writer(f)
#    writer.writerow(['id', 'title', 'focus_sentence', 'keywords', 'keyword_aspect', 'aspect_politics', 'aspect_humanity', 'aspect_science', 'aspect_economy'])  # comment off from the 2nd run!!
    for thread_id in cc_thread:
        submission_id, _ = thread_id.split('-')  # submissionID, delta_commentID
        for comment_dict in delta_thread_dict[thread_id]:
            comment_id = comment_dict['comment_id']
            comment = comment_dict['comment'].replace('\n\n', ' ')
            doc = nlp(comment)
            for sent_id, sent in enumerate(doc.sents):
                selected_id = f'{thread_id}-{comment_id}-{sent_id}'
                words = sent.text.split()
                if (MIN <= len(words) < MAX) and (contain_keyword(sent.text, ASPECT)):
                    if selected_id in set(selected_sent_ids):
                        aspect_count += 1
                        continue
                    else:
                        aspect_count += 1
                        keywords = get_keyword(sent.text, ASPECT)
                        writer.writerow([
                                        selected_id, 
                                        get_submission_title(submission_id), 
                                        sent.text.strip(),
                                        ', '.join(keywords),
                                        ASPECT
                        ])
                        selected_sent_ids.append(selected_id)

    for submission_id in cc_submission_ids.values:
        submission_body = get_submission_body(submission_id)
        doc = nlp(submission_body)
        for sent_id, sent in enumerate(doc.sents):
            selected_id = f'{submission_id}-{sent_id}'
            words = sent.text.split()
            if (MIN <= len(words) < MAX) and (contain_keyword(sent.text, ASPECT)):
                if selected_id in set(selected_sent_ids):
                    aspect_count += 1
                    continue
                else:
                    aspect_count += 1
                    keywords = get_keyword(sent.text, ASPECT)
                    writer.writerow([
                                      selected_id,
                                      get_submission_title(submission_id),
                                      sent.text.strip(),
                                      ', '.join(keywords),
                                        ASPECT,
                    ])
                    selected_sent_ids.append(selected_id)
print(aspect_count, "sentences about", ASPECT)
print(len(selected_sent_ids), "sentences so far.")

120 sentences about industry
722 sentences so far.
